<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Capstone проект №1. Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 6.  Vowpal Wabbit

На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных по посещению сайтов.

**План 6 недели:**
- Часть 1. Статья по Vowpal Wabbit
- Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов
 - 2.1. Подготовка данных
 - 2.2. Валидация по отложенной выборке
 - 2.3. Валидация по тестовой выборке (Public Leaderboard)

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
Также будет полезна [презентация](https://github.com/esokolov/ml-course-msu/blob/master/ML15/lecture-notes/Sem08_vw.pdf) лектора специализации Евгения Соколова. И, конечно же, [документация](https://github.com/JohnLangford/vowpal_wabbit/wiki) Vowpal Wabbit.

<font color='red'>**Задание:**</font> заполните код в этой тетрадке и выберите ответы в [веб-форме](https://docs.google.com/forms/d/1wteunpEhAt_9s-WBwxYphB6XpniXsAZiFSNuFNmvOdk). 

## Часть 1. Статья про Vowpal Wabbit
Прочитайте [статью](https://habrahabr.ru/company/ods/blog/326418/) про Vowpal Wabbit на Хабре из серии открытого курса OpenDataScience по машинному обучению. Материал для этой статьи зародился из нашей специализации. Скачайте [тетрадку](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_notebooks/topic08_sgd_hashing_vowpal_wabbit/topic8_sgd_hashing_vowpal_wabbit.ipynb), прилагаемую к статье, посмотрите код, изучите его, поменяйте, только так можно разобраться с Vowpal Wabbit.

## Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов

### 2.1. Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные [отсюда](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.**

In [1]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [72]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score, accuracy_score


In [2]:
# Поменяйте на свой путь к данным
# PATH_TO_DATA = '~/capstone_user_identification'
PATH_TO_DATA = 'data/capstone_user_identification'

**Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке. **

In [3]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

In [116]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')
#test_df_400.fillna(0, inplace=True).astype('int')

In [117]:
test_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,9,2014-10-04 12:24:43,304.0,2014-10-04 12:25:34,308.0,2014-10-04 12:28:33,307.0,2014-10-04 12:28:33,91.0,2014-10-04 12:28:33,308.0,2014-10-04 12:28:34,312.0,2014-10-04 12:30:31,300.0,2014-10-04 12:31:30,305.0,2014-10-04 12:32:31,309.0,2014-10-04 12:34:31
2,838,2014-12-02 09:20:37,504.0,2014-12-02 09:20:38,68.0,2014-12-02 09:20:38,11.0,2014-12-02 09:20:38,838.0,2014-12-02 09:20:38,11.0,2014-12-02 09:20:40,838.0,2014-12-02 09:20:41,886.0,2014-12-02 09:20:42,27.0,2014-12-02 09:20:42,305.0,2014-12-02 09:20:42
3,190,2014-10-01 09:27:38,192.0,2014-10-01 09:27:38,8.0,2014-10-01 09:27:38,189.0,2014-10-01 09:27:38,191.0,2014-10-01 09:27:39,189.0,2014-10-01 09:27:39,190.0,2014-10-01 09:27:39,2375.0,2014-10-01 09:27:39,192.0,2014-10-01 09:27:39,8.0,2014-10-01 09:27:39
4,295,2014-10-02 14:34:17,295.0,2014-10-02 14:34:18,295.0,2014-10-02 14:34:19,295.0,2014-10-02 14:34:20,295.0,2014-10-02 14:34:21,295.0,2014-10-02 14:34:22,295.0,2014-10-02 14:34:23,295.0,2014-10-02 14:34:24,295.0,2014-10-02 14:34:25,295.0,2014-10-02 14:34:26
5,31,2014-05-19 17:50:21,3177.0,2014-05-19 17:50:21,3174.0,2014-05-19 17:50:21,32434.0,2014-05-19 17:50:22,27.0,2014-05-19 17:50:22,31.0,2014-05-19 17:50:22,5698.0,2014-05-19 17:50:25,5698.0,2014-05-19 17:50:26,5698.0,2014-05-19 17:50:27,4173.0,2014-05-19 17:50:27


In [5]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

In [15]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [7]:
from sklearn import preprocessing

In [12]:
''' ВАШ КОД ЗДЕСЬ '''
y = train_df_400['user_id'].values
class_encoder = preprocessing.LabelEncoder()
y_for_vw = class_encoder.fit_transform(y) + 1

In [13]:
y_for_vw

array([262,  82,  16, ..., 254,  91, 256], dtype=int64)

**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**
- объедините обучающиую и тестовую выборки
- выберите только сайты (признаки от 'site1' до 'site10')
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат `csr_matrix`
- разбейте обратно на обучающую и тестовую части

In [14]:
sites = ['site' + str(i) for i in range(1, 11)]

In [21]:
''' ВАШ КОД ЗДЕСЬ '''
train_test_df_400 = pd.concat([train_df_400, test_df_400])
train_test_df_sites_400 = train_test_df_400[sites].fillna(0).astype('int')

train_test_sparse_400 = csr_matrix((np.ones(train_test_df_sites_400.values.size), 
                               train_test_df_sites_400.values.ravel(), 
                               [train_test_df_sites_400.values.shape[1] * i 
                                for i in range(train_test_df_sites_400.values.shape[0] + 1)]), dtype=int)[:, 1:]
X_train_sparse = train_test_sparse_400[:train_df_400.shape[0]]
X_test_sparse = train_test_sparse_400[train_df_400.shape[0]:]
y = train_df_400['user_id'].values

### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [53]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = train_df_400[sites].fillna(0).astype('int').iloc[:train_share, :]
valid_df = train_df_400[sites].fillna(0).astype('int').iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [54]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

**Реализуйте функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.**

Вход:
 - X – матрица `NumPy` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [55]:
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw'):
    ''' ВАШ КОД ЗДЕСЬ '''
    with open(out_file, 'w') as out_vw:

        for i in range(X.shape[0]):
            if train:
                target = y[i]
            else:
                target = 1

            s = '{} | {} {} {} {} {} {} {} {} {} {}\n'.format(
                 target, X[i, 0], X[i, 1], X[i, 2], X[i, 3], X[i, 4], X[i, 5], X[i, 6], X[i, 7], X[i, 8], X[i, 9])
            out_vw.write(s)

**Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора `NumPy`.**

In [56]:
train_df_part.values.shape

(127955, 10)

In [57]:
%%time
# будет 4 вызова
''' ВАШ КОД ЗДЕСЬ '''
arrays_to_vw(train_df_part.values, y_train_part_for_vw, train=True, out_file=os.path.join(PATH_TO_DATA, 'train_part.vw'))
arrays_to_vw(valid_df.values, y_valid_for_vw, train=True, out_file=os.path.join(PATH_TO_DATA, 'valid.vw'))
arrays_to_vw(train_df_400[sites].values, y_for_vw, train=True, out_file=os.path.join(PATH_TO_DATA, 'train.vw'))
arrays_to_vw(test_df_400[sites].values, train=False, out_file=os.path.join(PATH_TO_DATA, 'test.vw'))

Wall time: 4.95 s


**Результат должен получиться таким.**

In [58]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [7]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [105]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [8]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [102]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9.0 304.0 308.0 307.0 91.0 308.0 312.0 300.0 305.0 309.0
1 | 838.0 504.0 68.0 11.0 838.0 11.0 838.0 886.0 27.0 305.0
1 | 190.0 192.0 8.0 189.0 191.0 189.0 190.0 2375.0 192.0 8.0


In [9]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


**Обучите модель Vowpal Wabbitна выборке `train_part.vw`. Укажите, что решается задача классификации с 400 классами (`--oaa`), сделайте 3 прохода по выборке (`--passes`). Задайте некоторый кэш-файл (`--cache_file`, можно просто указать флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажите значение параметра `b`=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите `random_seed`=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [61]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [63]:
%%time
''' ВАШ КОД ЗДЕСЬ '''
# обучаем

!vw \
-d $train_part_vw \
-f $model \
--oaa 400 \
-c \
--passes 3 \
-k \
-b 26 \
--random_seed=17 \

Wall time: 59.4 s


final_regressor = data/capstone_user_identification\vw_model.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = data/capstone_user_identification\train_part.vw.cache
Reading datafile = data/capstone_user_identification\train_part.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976563 0.984375          

**Запишите прогнозы на выборке *valid.vw* в *vw_valid_pred.csv*.**

In [64]:
%%time
''' ВАШ КОД ЗДЕСЬ '''
# прогноз на выборке valid.vw* в *vw_valid_pred.csv

!vw \
-i $model -t \
-d $valid_vw \
-p $PATH_TO_DATA/vw_valid_pred.csv \

Wall time: 1.26 s


only testing
predictions = data/capstone_user_identification/vw_valid_pred.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/capstone_user_identification\valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4      188       11
1.000000 1.000000            2            2.0      160      220       11
0.750000 0.500000            4            4.0      143      143       11
0.750000 0.750000            8            8.0      247      247       11
0.687500 0.625000           16           16.0      341       30       11
0.593750 0.500000           32           32.0      237      237       11
0.609375 0.625000           64           64.0      178      178       11
0.640625 0.671875          128          128.0      132      228       11
0.656250 0.671875          25

**Считайте прогнозы *kaggle_data/vw_valid_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [69]:
''' ВАШ КОД ЗДЕСЬ '''
vw_valid_pred_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'vw_valid_pred.csv'), header=None)

In [70]:
vw_valid_pred_df.head()

,0
0,188
1,220
2,364
3,143
4,132


In [74]:
accuracy_score(vw_valid_pred_df, y_valid_for_vw)

0.34541741128414605

**Теперь обучите `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделайте прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитайте доли верных ответов. Логистическая регрессия будет обучаться долго (у меня – 15 минут) – это нормально. Укажите везде `random_state`=17, `n_jobs`=-1.**

In [76]:
''' ВАШ КОД ЗДЕСЬ '''
logit = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(loss='log', n_iter=3, random_state=17, n_jobs=-1)

In [77]:
%%time
''' ВАШ КОД ЗДЕСЬ '''
logit.fit(X_train_part_sparse, y_train_part)

Wall time: 31min 59s


In [78]:
%%time
''' ВАШ КОД ЗДЕСЬ '''
sgd_logit.fit(X_train_part_sparse, y_train_part)

Wall time: 12.3 s


**<font color='red'>Вопрос 1. </font> Посчитайте долю правильных ответов на отложенной выборке для Vowpal Wabbit, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 2. </font> Посчитайте долю правильных ответов на отложенной выборке для SGD, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 3. </font> Посчитайте долю правильных ответов на отложенной выборке для логистической регрессии, округлите до 3 знаков после запятой.**

In [80]:
''' ВАШ КОД ЗДЕСЬ '''
vw_valid_acc = accuracy_score(vw_valid_pred_df, y_valid_for_vw)
logit_valid_acc = accuracy_score(logit.predict(X_valid_sparse), y_valid)
sgd_valid_acc = accuracy_score(sgd_logit.predict(X_valid_sparse), y_valid)
print("vw_valid_acc = {}, \nsgd_valid_acc = {}, \nlogit_valid_acc = {}".format(round(vw_valid_acc, 3), round(sgd_valid_acc, 3),
                                     round(logit_valid_acc, 3)))

vw_valid_acc = 0.345, 
sgd_valid_acc = 0.291, 
logit_valid_acc = 0.363


In [ ]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [ ]:
write_answer_to_file("{}  {} {}".format(round(vw_valid_acc, 3), round(sgd_valid_acc, 3),
                                     round(logit_valid_acc, 3)),  
                     'answer6_1.txt')

### 2.3. Валидация по тестовой выборке (Public Leaderboard)

**Обучите модель VW с теми же параметрами на всей обучающей выборке – *train.vw*.**

In [81]:
%%time
!vw \
-d $train_vw \
-f $PATH_TO_DATA/vw_model_train.vw \
--oaa 400 \
-c \
--passes 3 \
-k \
-b 26 \
--random_seed=17 \

Wall time: 1min 3s


final_regressor = data/capstone_user_identification/vw_model_train.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = data/capstone_user_identification\train.vw.cache
Reading datafile = data/capstone_user_identification\train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976563 0.984375          128 

**Сделайте прогноз для тестовой выборки.**

In [82]:
%%time
!vw \
-i $PATH_TO_DATA/vw_model_train.vw -t \
-d $train_vw \
-p $PATH_TO_DATA/vw_train_pred.csv \

Wall time: 3.63 s


only testing
predictions = data/capstone_user_identification/vw_train_pred.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/capstone_user_identification\train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0      262      262       11
0.000000 0.000000            2            2.0       82       82       11
0.500000 1.000000            4            4.0      241      212       11
0.625000 0.750000            8            8.0      352      352       11
0.750000 0.875000           16           16.0      156      354       11
0.656250 0.562500           32           32.0      243      243       11
0.562500 0.468750           64           64.0       79       79       11
0.570313 0.578125          128          128.0      298      194       11
0.550781 0.531250          25

**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.**

In [83]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [85]:
''' ВАШ КОД ЗДЕСЬ '''
vw_pred = class_encoder.inverse_transform(pd.read_csv(os.path.join(PATH_TO_DATA, 'vw_train_pred.csv'), header=None).values - 1)

In [86]:
write_to_submission_file(vw_pred, os.path.join(PATH_TO_DATA, 'vw_400_users.csv'))

In [90]:
vw_400_users = pd.read_csv(os.path.join(PATH_TO_DATA, 'vw_400_users.csv'), header=None)

In [91]:
vw_400_users.head()

,0,1
0,session_id,user_id
1,1,653
2,2,198
3,3,635
4,4,541


In [92]:
vw_400_users.shape

(182794, 2)

**Сделайте то же самое для SGD и логистической регрессии. Тут уже ждать обучение логистической регрессии совсем скучно (заново запускать тетрадку вам не захочется), но давайте дождемся.**

In [87]:
''' ВАШ КОД ЗДЕСЬ '''
logit_test_pred = logit.fit(X_train_sparse, y).predict(X_test_sparse)
sgd_logit_test_pred = sgd_logit.fit(X_train_sparse, y).predict(X_test_sparse)

In [88]:
write_to_submission_file(logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'logit_400_users.csv'))
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'sgd_400_users.csv'))

In [93]:
logit_400_users = pd.read_csv(os.path.join(PATH_TO_DATA, 'logit_400_users.csv'), header=None)

In [98]:
logit_400_users[-5:]

,0,1
46469,46469,213
46470,46470,7
46471,46471,656
46472,46472,751
46473,46473,179


Посмотрим на доли правильных ответов на публичной части (public leaderboard) тестовой выборки [этого](https://inclass.kaggle.com/c/identify-me-if-you-can4) соревнования.

**<font color='red'>Вопрос 4. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для Vowpal Wabbit?**

**<font color='red'>Вопрос 5. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для SGD?**

**<font color='red'>Вопрос 6. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для логистической регрессии?**


**В заключение по заданию:**
- Про соотношение качества классификации и скорости обучения VW, SGD и logit выводы предлагается сделать самостоятельно
- Пожалуй, задача классификации на 400 классов (идентификация 400 пользователей) решается недостаточно хорошо при честном отделении по времени тестовой выборки от обучающей. Далее мы будем соревноваться в идентификации одного пользователя (Элис) – [вот](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнование, в котором предлагается поучаствовать. Не перепутайте! 

**Удачи!**